In [16]:
import os

In [17]:
from bs4 import BeautifulSoup
import cloudscraper
import geopandas as gpd
import numpy as np
import pandas as pd

In [18]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [19]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [20]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [21]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [22]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [23]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Carls Jr Data

In [24]:
scraper = cloudscraper.create_scraper()

In [74]:
store_list = []
url_base = "https://locations.carlsjr.com/"
r = scraper.get(url_base)
states_soup = BeautifulSoup(r.text, "html.parser")
states_a = states_soup.find_all("a", {"class": "Directory-listLink"})
for state_a in tqdm(states_a, desc="Parsing States"):
    state_name = state_a.text
    state_url = state_a.attrs["href"]
    state_code = states_with_population_df[
        states_with_population_df["NAME"] == state_name
    ].iloc[0]["STUSPS"]

    # Get State code
    state_store_list = []

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    # Get all locations in a state
    r_state_locations = scraper.get(state_url)

    if r_state_locations.status_code != 200:
        print(state_code)
        continue

    state_locations_soup = BeautifulSoup(r_state_locations.text, "html.parser")
    locations_as = state_locations_soup.find_all("a", {"class": "Directory-listLink"})

    # Parse all locations in a state
    for locations_a in tqdm(locations_as, desc=f"Parsing Locations In {state_code}"):
        location_url = locations_a.attrs["href"]

        r_location = scraper.get(location_url)
        location_soup = BeautifulSoup(r_location.text, "html.parser")
        store_as = location_soup.find_all("a", {"class": "Teaser-ctaLink"})

        # Parse through all stores
        for store_a in store_as:
            store_url = store_a.attrs["href"]
            r_store = scraper.get(store_url)
            store_soup = BeautifulSoup(r_store.text, "html.parser")

            try:
                lat = float(
                    store_soup.find(
                        "meta", {"property": "place:location:latitude"}
                    ).attrs["content"]
                )
                lon = float(
                    store_soup.find(
                        "meta", {"property": "place:location:longitude"}
                    ).attrs["content"]
                )
            except Exception as e:
                print(e)
                print(store_url)
            point = Point(lon, lat)
            store_dict = {
                "STATE": state_code,
                "geometry": point,
            }
            store_list.append(store_dict)
            state_store_list.append(store_dict)

    if state_store_list:
        carls_jr_state_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        carls_jr_state_gdf.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/16 [00:00<?, ?it/s]

Parsing Locations In CA:   0%|          | 0/321 [00:00<?, ?it/s]

Parsing Locations In CO:   0%|          | 0/29 [00:00<?, ?it/s]

Parsing Locations In FL:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing Locations In HI:   0%|          | 0/2 [00:00<?, ?it/s]

...

Parsing Locations In OR:   0%|          | 0/32 [00:00<?, ?it/s]

Parsing Locations In TX:   0%|          | 0/19 [00:00<?, ?it/s]

Parsing Locations In UT:   0%|          | 0/33 [00:00<?, ?it/s]

Parsing Locations In WA:   0%|          | 0/27 [00:00<?, ?it/s]

Parsing Locations In WY:   0%|          | 0/3 [00:00<?, ?it/s]

#### If the above cells breaks just keep running until complete. Then run the line below

In [75]:
carls_jr_state_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        state_carls_jr_gdf = gpd.read_file(constructed_path)
        carls_jr_state_gdfs.append(state_carls_jr_gdf)

In [76]:
carls_jr_state_gdf = gpd.GeoDataFrame(pd.concat(carls_jr_state_gdfs, ignore_index=True))

In [78]:
carls_jr_state_gdf = carls_jr_state_gdf.to_crs(9311)
carls_jr_state_gdf.to_file(f"data/carlsjr.gpkg")

## Combine With States

In [128]:
carls_jr_state_counts_series = carls_jr_state_gdf.groupby("STATE").size()
carls_jr_state_state_counts_df = pd.DataFrame(
    {
        "STUSPS": carls_jr_state_counts_series.index,
        "stores": carls_jr_state_counts_series.values,
    }
)

In [129]:
carls_jr_state_state_counts_df["STUSPS"] = carls_jr_state_state_counts_df[
    "STUSPS"
].str.upper()

In [130]:
carls_jr_state_state_counts_gdf = states_with_population_df.merge(
    carls_jr_state_state_counts_df, on="STUSPS", how="left"
)

In [131]:
carls_jr_state_state_counts_gdf = carls_jr_state_state_counts_gdf.fillna(0)
carls_jr_state_state_counts_gdf["stores"] = carls_jr_state_state_counts_gdf[
    "stores"
].astype(int)

In [132]:
carls_jr_state_state_counts_gdf["per_1000"] = carls_jr_state_state_counts_gdf[
    "stores"
] / (carls_jr_state_state_counts_gdf["POPULATION"] / 1000)
carls_jr_state_state_counts_gdf["per_10k"] = carls_jr_state_state_counts_gdf[
    "stores"
] / (carls_jr_state_state_counts_gdf["POPULATION"] / 10_000)
carls_jr_state_state_counts_gdf["per_100k"] = (
    carls_jr_state_state_counts_gdf["stores"]
    / (carls_jr_state_state_counts_gdf["POPULATION"] / 100000)
).round(decimals=2)

carls_jr_state_state_counts_gdf["per_500k"] = (
    carls_jr_state_state_counts_gdf["stores"]
    / (carls_jr_state_state_counts_gdf["POPULATION"] / 500_000)
).round(decimals=2)

carls_jr_state_state_counts_gdf["per_1m"] = (
    carls_jr_state_state_counts_gdf["stores"]
    / (carls_jr_state_state_counts_gdf["POPULATION"] / 1_000_000)
).round(decimals=2)

carls_jr_state_state_counts_gdf["people_per_store"] = (
    carls_jr_state_state_counts_gdf["POPULATION"]
    / carls_jr_state_state_counts_gdf["stores"]
).round(decimals=2)

In [133]:
carls_jr_state_state_counts_gdf.replace([np.inf, -np.inf], 0, inplace=True)

In [135]:
carls_jr_state_state_counts_gdf = carls_jr_state_state_counts_gdf.dropna()

In [136]:
carls_jr_state_state_counts_gdf["people_per_store"] = carls_jr_state_state_counts_gdf[
    "people_per_store"
].astype(int)

In [137]:
carls_jr_state_state_counts_gdf = carls_jr_state_state_counts_gdf.dropna()

In [138]:
carls_jr_state_state_counts_gdf = carls_jr_state_state_counts_gdf.to_crs(9311)
carls_jr_state_state_counts_gdf.to_file(f"data/carls_jrs_per_state.gpkg")